In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('tmdb_5000_movies.csv')

In [64]:
df.head()

,title,vote_count,vote_average,runtime,genres,overview,score,cluster
0,Avatar,11800,7.2,162.0,"Action,Adventure,Fantasy,Science Fiction","In the 22nd century, a paraplegic Marine is di...",7846.981934,10
1,Pirates of the Caribbean: At World's End,4500,6.9,169.0,"Adventure,Fantasy,Action","Captain Barbossa, long believed to be dead, ha...",3021.510742,4
2,Spectre,4466,6.3,148.0,"Action,Adventure,Crime",A cryptic message from Bond’s past sends him o...,2993.343750,3
3,The Dark Knight Rises,9106,7.6,165.0,"Action,Crime,Drama,Thriller",Following the death of District Attorney Harve...,6066.297852,8
4,John Carter,2124,6.1,132.0,"Action,Adventure,Science Fiction","John Carter is a war-weary, former military ca...",1440.288818,0


In [4]:
df.shape

(4803, 20)

In [5]:
df.isnull().sum()

budget                     0
genres                     0
homepage                3091
id                         0
keywords                   0
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
dtype: int64

In [6]:
df = df.drop('homepage',axis = 1)

In [7]:
df = df.drop('tagline',axis = 1)

In [8]:
import ast

In [9]:

def extract_genres(df):
    for i in range(0,4803):
        l = []
        res = ast.literal_eval(df.genres[i])
        for item in res:
            l.append(item['name'])
        ans = ','.join([elem for elem in l]) 
        df.genres[i] = ans
extract_genres(df)

C:\Users\moksh\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [10]:
columns = ['title','vote_count','vote_average','runtime','genres','overview']

In [11]:
df = df[columns]

In [12]:
df.head()

,title,vote_count,vote_average,runtime,genres,overview
0,Avatar,11800,7.2,162.0,"Action,Adventure,Fantasy,Science Fiction","In the 22nd century, a paraplegic Marine is di..."
1,Pirates of the Caribbean: At World's End,4500,6.9,169.0,"Adventure,Fantasy,Action","Captain Barbossa, long believed to be dead, ha..."
2,Spectre,4466,6.3,148.0,"Action,Adventure,Crime",A cryptic message from Bond’s past sends him o...
3,The Dark Knight Rises,9106,7.6,165.0,"Action,Crime,Drama,Thriller",Following the death of District Attorney Harve...
4,John Carter,2124,6.1,132.0,"Action,Adventure,Science Fiction","John Carter is a war-weary, former military ca..."


In [220]:
#df['vote_average'] = pd.to_numeric(df['vote_average'],downcast = 'float')

In [221]:
#df['runtime'] = pd.to_numeric(df['runtime'],downcast = 'float')

In [222]:
#df['vote_count'] = pd.to_numeric(df['vote_count'],downcast = 'float')

In [223]:
#df.head()

In [13]:
def imp_df(column_names, importances):
    data = {
        'Feature': column_names,
        'Importance': importances,
    }
    df = pd.DataFrame(data) \
        .set_index('Feature') \
        .sort_values('Importance', ascending=False)

    return df


In [14]:

from sklearn.base import clone
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

In [15]:
def drop_col_feat_imp(model, X_train, y_train, random_state=42):
    model_clone = clone(model)
    model_clone.random_state = random_state

    model_clone.fit(X_train, y_train)
    benchmark_score = model_clone.score(X_train, y_train)

    importances = []

    for col in X_train.columns:
        model_clone = clone(model)
        model_clone.random_state = random_state
        model_clone.fit(X_train.drop(col, axis=1), y_train)
        drop_col_score = model_clone.score(X_train.drop(col, axis=1), y_train)
        importances.append(benchmark_score - drop_col_score)

    return imp_df(X_train.columns, importances)

In [16]:
df.dropna()
X = df.drop(['vote_average', 'title', 'genres','overview'], axis=1)
y = df['vote_average']


In [17]:
X.loc[pd.isna(X["runtime"]), :].index

Int64Index([2656, 4140], dtype='int64')

In [18]:
y = pd.DataFrame(y)

In [19]:
y.loc[pd.isna(y["vote_average"]), :].index

Int64Index([], dtype='int64')

In [20]:
X.fillna(X.mean(), inplace=True)

In [21]:
X.loc[pd.isna(X["runtime"]), :].index

Int64Index([], dtype='int64')

In [22]:
#y_train.isna().count()

In [23]:
#X_train.isna().count()

In [24]:
X=X.astype("float32")

In [25]:
y = y.astype("float32")

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.33, random_state=0)

In [238]:
#df['runtime'] = pd.to_numeric(df['runtime'],downcast = 'float')

In [28]:
type(y["vote_average"][0])

numpy.float32

In [29]:
rf = RandomForestRegressor(n_estimators=10, random_state = 42)

In [30]:
rf.fit(X_train,y_train)

C:\Users\moksh\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=10, n_jobs=None, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)

In [31]:
rf.score(X_train, y_train)

0.8500711746037886

In [32]:
def imp_df(column_names, importances):
    data = {
        'Feature': column_names,
        'Importance': importances,
    }
    df = pd.DataFrame(data) \
        .set_index('Feature') \
        .sort_values('Importance', ascending=False)
    
    return df

In [33]:
def drop_col_feat_imp(model, X_train, y_train, random_state=42):
    model_clone = clone(model)
    model_clone.random_state = random_state
    
    model_clone.fit(X_train, y_train)
    benchmark_score = model_clone.score(X_train, y_train)
    
    importances = []
    
    for col in X_train.columns:
        model_clone = clone(model)
        model_clone.random_state = random_state
        model_clone.fit(X_train.drop(col, axis=1), y_train)
        drop_col_score = model_clone.score(X_train.drop(col, axis=1), y_train)
        importances.append(benchmark_score - drop_col_score)
    
    return imp_df(X_train.columns, importances)

In [34]:
drop_col_feat_imp(rf, X_train, y_train)

C:\Users\moksh\anaconda3\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """
C:\Users\moksh\anaconda3\lib\site-packages\ipykernel_launcher.py:13: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  del sys.path[0]
C:\Users\moksh\anaconda3\lib\site-packages\ipykernel_launcher.py:13: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  del sys.path[0]


,Importance
Feature,
vote_count,0.661283
runtime,0.270634


In [35]:
df['score'] = (
    0.661283 * df['vote_count'] + 0.270634 * df['runtime']
)


In [36]:
df.head()

,title,vote_count,vote_average,runtime,genres,overview,score
0,Avatar,11800,7.2,162.0,"Action,Adventure,Fantasy,Science Fiction","In the 22nd century, a paraplegic Marine is di...",7846.982108
1,Pirates of the Caribbean: At World's End,4500,6.9,169.0,"Adventure,Fantasy,Action","Captain Barbossa, long believed to be dead, ha...",3021.510646
2,Spectre,4466,6.3,148.0,"Action,Adventure,Crime",A cryptic message from Bond’s past sends him o...,2993.343710
3,The Dark Knight Rises,9106,7.6,165.0,"Action,Crime,Drama,Thriller",Following the death of District Attorney Harve...,6066.297608
4,John Carter,2124,6.1,132.0,"Action,Adventure,Science Fiction","John Carter is a war-weary, former military ca...",1440.288780


In [37]:
df = df.reset_index(drop=True)

In [38]:
df.sort_values(by='score', ascending=False).head(20)

,title,vote_count,vote_average,runtime,genres,overview,score
96,Inception,13752,8.1,148.0,"Action,Thriller,Science Fiction,Mystery,Adventure","Cobb, a skilled thief who commits corporate es...",9134.017648
65,The Dark Knight,12002,8.2,152.0,"Drama,Action,Crime,Thriller",Batman raises the stakes in his war on crime. ...,7977.854934
0,Avatar,11800,7.2,162.0,"Action,Adventure,Fantasy,Science Fiction","In the 22nd century, a paraplegic Marine is di...",7846.982108
16,The Avengers,11776,7.4,143.0,"Science Fiction,Action,Adventure",When an unexpected enemy emerges and threatens...,7825.969270
788,Deadpool,10995,7.4,108.0,"Action,Adventure,Comedy",Deadpool tells the origin story of former Spec...,7300.035057
95,Interstellar,10867,8.1,169.0,"Adventure,Drama,Science Fiction",Interstellar chronicles the adventures of a gr...,7231.899507
287,Django Unchained,10099,7.8,165.0,"Drama,Western","With the help of a German bounty hunter, a fre...",6722.951627
94,Guardians of the Galaxy,9742,7.9,121.0,"Action,Science Fiction,Adventure","Light years from Earth, 26 years after being a...",6474.965700
426,The Hunger Games,9455,6.9,142.0,"Science Fiction,Adventure,Fantasy",Every year in the ruins of what was once North...,6290.860793
127,Mad Max: Fury Road,9427,7.2,120.0,"Action,Adventure,Science Fiction,Thriller",An apocalyptic story set in the furthest reach...,6266.390921


In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()
matrix = tfidf_vectorizer.fit_transform(df['genres'])

In [42]:
from scipy.signal import argrelextrema
from sklearn.neighbors.kde import KernelDensity


C:\Users\moksh\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.neighbors.kde module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [45]:
df['score'] = df['score'].astype('float32')

In [46]:
df.head()

,title,vote_count,vote_average,runtime,genres,overview,score
0,Avatar,11800,7.2,162.0,"Action,Adventure,Fantasy,Science Fiction","In the 22nd century, a paraplegic Marine is di...",7846.981934
1,Pirates of the Caribbean: At World's End,4500,6.9,169.0,"Adventure,Fantasy,Action","Captain Barbossa, long believed to be dead, ha...",3021.510742
2,Spectre,4466,6.3,148.0,"Action,Adventure,Crime",A cryptic message from Bond’s past sends him o...,2993.343750
3,The Dark Knight Rises,9106,7.6,165.0,"Action,Crime,Drama,Thriller",Following the death of District Attorney Harve...,6066.297852
4,John Carter,2124,6.1,132.0,"Action,Adventure,Science Fiction","John Carter is a war-weary, former military ca...",1440.288818


In [66]:
vals.shape

(4803, 1)

In [69]:
df['score'].shape

(4803,)

In [52]:
vals = df['score'].values.reshape(-1, 1)
kde = KernelDensity(kernel='gaussian', bandwidth=3).fit(vals)

s = np.linspace(650, 20000)

e = kde.score_samples(s.reshape(-1,1))

In [49]:
df.loc[pd.isna(df["score"]), :].index

Int64Index([2656, 4140], dtype='int64')

In [50]:
df.fillna(df.mean(), inplace=True)

In [51]:
df.loc[pd.isna(df["score"]), :].index

Int64Index([], dtype='int64')

In [54]:
mi, ma = argrelextrema(e, np.less)[0], argrelextrema(e, np.greater)[0]

In [55]:
points = np.concatenate((s[mi], s[ma]), axis=0)
buckets = []

for point in points:
    buckets.append(point)

In [56]:
buckets = np.array(buckets)
buckets.sort()

In [57]:
df['cluster'] = buckets.searchsorted(df.score)

In [58]:
df.groupby('cluster').size()

cluster
0     4515
1       84
2       47
3       43
4       32
5       15
6       34
7       13
8       14
9        2
10       3
12       1
dtype: int64

In [61]:
from sklearn.metrics.pairwise import linear_kernel

In [87]:
def get_recommendations(movie_index):
    print(df.iloc[movie_index])
    
    kernel = linear_kernel(matrix[movie_index], matrix)
    sim_scores = list(enumerate(kernel[0]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    indexes = [i[0] for i in sim_scores if i[0] != movie_index and i[1] > .5]
    cond1 = (df.index.isin(indexes))
    cond2 = (df.cluster == df.iloc[movie_index]['cluster'])
    
    
    selected = df.loc[cond1 & cond2] \
        .sort_values(by='score', ascending=False).head(20)
    display(selected)

In [88]:
get_recommendations(545)

title                                                 Unbreakable
vote_count                                                   1946
vote_average                                                  6.9
runtime                                                       106
genres                             Science Fiction,Thriller,Drama
overview        An ordinary man makes an extraordinary discove...
score                                                     1315.54
cluster                                                         0
Name: 545, dtype: object


,title,vote_count,vote_average,runtime,genres,overview,score,cluster
257,Real Steel,2692,6.6,127.0,"Action,Science Fiction,Drama","In the near-future, Charlie Kenton is a washed...",1814.544312,0
1506,Source Code,2699,7.1,93.0,"Thriller,Science Fiction,Mystery",Decorated soldier Captain Colter Stevens wakes...,1809.971802,0
238,Teenage Mutant Ninja Turtles,2636,5.8,101.0,"Science Fiction,Action,Adventure,Fantasy,Comedy",The city needs heroes. Darkness has settled ov...,1770.476074,0
275,Minority Report,2608,7.1,145.0,"Action,Thriller,Science Fiction,Mystery",John Anderton is a top 'Precrime' cop in the l...,1763.868042,0
1187,Bridge of Spies,2583,7.2,141.0,"Thriller,Drama","During the Cold War, the Soviet Union captures...",1746.253418,0
56,Star Trek Beyond,2568,6.6,122.0,"Action,Adventure,Science Fiction",The USS Enterprise crew explores the furthest ...,1731.192139,0
193,After Earth,2532,5.0,100.0,"Science Fiction,Action,Adventure",One thousand years after cataclysmic events fo...,1701.432007,0
362,Pixels,2513,5.6,105.0,"Action,Comedy,Science Fiction",Video game experts are recruited by the milita...,1690.220703,0
149,Armageddon,2482,6.4,151.0,"Action,Thriller,Science Fiction,Adventure",When an asteroid threatens to collide with Ear...,1682.170166,0
91,Independence Day: Resurgence,2491,4.9,120.0,"Action,Adventure,Science Fiction",We always knew they were coming back. Using re...,1679.732056,0
